In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [0]:
root_loc = '/Volumes/workspace/default/fandc_data'

schema = StructType([
    StructField("Date", StringType(), False),
    StructField(" Type",StringType(), True),
    StructField(" Description", StringType(), True),
    StructField(" Category", StringType(), True),
    StructField(" Amount", StringType(), True),
    StructField(" Balance", DoubleType(), True),
    StructField(" Note", StringType(), True)
    ])

In [0]:
df = (
  spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("rescuedDataColumn", "_rescued_data")
    .option("header", "true")
    .option("quote", "")
    .option("sep", ",")
    .schema(schema)
    .load(root_loc)
)
df_clean = df.selectExpr(
    "`Date` as Date",
    "` Type` as Type",
    "` Description` as Description",
    "` Category` as Category",
    "` Amount` as Amount",
    "` Balance` as Balance",
    "` Note` as Note",
    '_rescued_data'
)

In [0]:
(
  df_clean.writeStream
    .option("checkpointLocation", "/Volumes/workspace/default/checkpoints/fandc_data_checkpoint")
    .trigger(availableNow=True)
    .toTable("workspace.default.fandc_raw_data")
)